# Multi Label classification with fast.ai2


<Description>
    
    
In this kernel, I will briefly explain the background knowledge required to understand the dataset. Then, I will visualize the data with Plotly and Matplotlib.

<font size=3 color="red">Please upvote this kernel if you like it. It motivates me to produce more quality content :)</font>

# Contents

* [<font size=4>Background</font>](#1)
    * [Electrophysiology](#1.1)
    * [Clamp method](#1.2)
    * [Ion channels](#1.3)


* [<font size=4>EDA</font>](#2)
    * [Preparing the ground](#2.1)
    * [Signal data](#2.2)
    * [Denoising](#2.3)
    * [Open channels](#2.4)
    * [Measures of complexity (entropy and fractal dimension)](#2.5)


* [<font size=4>Ending Note</font>](#3)

In [ ]:
! pip install fastai2

In [ ]:
from fastai2.vision.all import *
from fastai2.data.all import *
from fastai2.callback.all import *

from torchvision.models import resnet50

In [ ]:
plant_source = untar_data(URLs.PLANET_SAMPLE)
df = pd.read_csv(plant_source/'labels.csv')

In [ ]:
df.head()

In [ ]:
########
# data block
batch_tfms = aug_transforms(flip_vert=True, max_lighting=0.1, max_zoom=1.05, max_warp=0.)

planet = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                  get_x=ColReader(0, pref=f"{plant_source}/train/", suff='.jpg'),
                  splitter=RandomSplitter(),
                  get_y=ColReader(1, label_delim=' '),
                   batch_tfms=batch_tfms
                  )

In [ ]:
dls = planet.dataloaders(df)
dls.show_batch()

In [ ]:
path = untar_data(URLs.PETS)
fnames = get_image_files(path/'images')

pat = r'([^/]+)_\d+.*$'
re.search(pat, str(fnames[0])).group(1)

In [ ]:
splitter = RandomSplitter(valid_pct=0.2)
splitter(fnames)

In [ ]:
item_tfms = [Resize(224, method='crop')]
batch_tfms = [*aug_transforms(size=256), Normalize.from_stats(*imagenet_stats)]

In [ ]:
block = DataBlock(blocks=(ImageBlock, CategoryBlock),
                 get_items=get_image_files,
                 get_y=RegexLabeller(pat),
                 splitter=splitter,
                 item_tfms=item_tfms,
                 batch_tfms=batch_tfms)

In [ ]:
dls = block.dataloaders(path, bs=128)

In [ ]:
dls = ImageDataLoaders.from_name_re(path, fnames, pat, item_tfms=item_tfms,
                                            batch_tfms=batch_tfms, bs=64)
dls.show_batch()

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=accuracy)

learn.fine_tune(5)

In [ ]:
###############
# Lambdas

blocks = (ImageBlock, MultiCategoryBlock)
get_x = lambda x: plant_source/'train'/f'{x[0]}.jpg'
get_y = lambda x: x[1].split(' ')
val = df.values[0]
get_x(df.values[0])

In [ ]:
planet = DataBlock(blocks=blocks, get_x=get_x, get_y=get_y, splitter=RandomSplitter(),
                  batch_tfms=batch_tfms)

In [ ]:
dls = planet.dataloaders(df)
dls.show_batch(max_n=9, figsize=(12,9))

In [ ]:
##################
# 3: Custom get_items Functions

def _planet_items(x): return (
    f'{plant_source}/train/'+x.image_name+'.jpg', x.tags.str.split())

In [ ]:
planet = DataBlock.from_columns(blocks=blocks,
                               get_items=_planet_items,
                               splitter=RandomSplitter(),
                               batch_tfms=batch_tfms)

In [ ]:
dls = planet.dataloaders(df)
dls.show_batch(max_n=9, figsize=(12,9))

## Training Models

In [ ]:
from fastai2.metrics import accuracy_multi

learn = cnn_learner(dls, resnet50, pretrained=True, metrics=[accuracy_multi])

In [ ]:
class BCEWithLogitsLossFlat(BaseLoss):
    "Same as `nn.CrossEntropyLoss`, but flattens input and target."
    def __init__(self, *args, axis=-1, floatify=True, thresh=0.5, **kwargs):
        super().__init__(nn.BCEWithLogitsLoss, *args, axis=axis, floatify=floatify, is_2d=False, **kwargs)
        self.thresh = thresh

    def decodes(self, x):    return x>self.thresh
    def activation(self, x): return torch.sigmoid(x)

In [ ]:
learn.loss_func = BCEWithLogitsLossFlat()
learn.lr_find()

In [ ]:
lr = 1e-2
learn = learn.to_fp16()
learn.fit_one_cycle(5, slice(lr))
learn.save('stage-1')

In [ ]:
learn.unfreeze()
learn.lr_find()

In [ ]:
learn.show_results()

In [ ]:
learn.fit_one_cycle(5, slice(1e-5, lr/5))

In [ ]:
learn.show_results(figsize=(15,15))

## Acknowledgment

# Ending note <a id="3"></a>

<font size=4 color="red">This concludes my EDA kernel. Please upvote this kernel if you like it. It motivates me to produce more quality content :)</font>